In [15]:
 # -*- coding: utf-8 -*-
import pyes
import elasticsearch
import json

In [2]:
from datetime import datetime,time

In [3]:
# Connecting to elasticsearch server
es_address='140.118.109.114:9200'
conn = pyes.es.ES(es_address)
conn.default_indices=["facebook_nested"]

In [4]:
# doc_types: fanpage,post,like,comment
def query(fid,doc_types):
    tq = pyes.query.TermQuery(field="fid",value=fid)
    qsearch = pyes.query.Search(tq,size=5000)
    result = conn.search(query=qsearch , doc_types=doc_types,size=5000)
    return result

In [5]:
def query_agg(fid,doc_types,agg_field):
    tq = pyes.query.TermQuery(field="fid",value=fid)
    tagg_comment = pyes.aggs.TermsAgg('genres',field=agg_field,size=0)
    qsearch = pyes.query.Search(tq)
    qsearch.agg.add(tagg_comment)
    result = conn.search(query=qsearch , doc_types=doc_types)
    return result

In [6]:
def get_share_count(fid_pid):
    tq = pyes.query.TermQuery(field="_id",value=fid_pid)
    qsearch = pyes.query.Search(tq)
    result = conn.search(query=qsearch , doc_types="post")
    try:
        return result.hits[0]["_source"]['shares']['count']
    except: 
        return 0

In [7]:
def get_created_time(fid_pid):
    tq = pyes.query.TermQuery(field="_id",value=fid_pid)
    qsearch = pyes.query.Search(tq)
    result = conn.search(query=qsearch , doc_types="post")
    return result.hits[0]["_source"]['created_time']

In [8]:
def get_type(fid_pid):
    tq = pyes.query.TermQuery(field="_id",value=fid_pid)
    qsearch = pyes.query.Search(tq)
    result = conn.search(query=qsearch , doc_types="post")
    try:
        return result.hits[0]["_source"]['type']
    except:
        return 0

In [9]:
# 輸入fid_pid 回傳link
def has_link(fid_pid):
    bq = pyes.query.BoolQuery() 
    m = pyes.query.TermQuery(field="_id",value=fid_pid)
    s1 = pyes.query.MatchQuery("message","https://")
    s2 = pyes.query.MatchQuery("message","http://")
    bq.add_must(m)
    bq.add_should(s1)
    bq.add_should(s2)
    es_result = conn.search(query=bq , doc_types='post')
    if es_result.total >0:
        return 1
    else:
        return 0

In [10]:
# initialize fanpage object with fanpage id and fanpage source from es/fanpage
class fanpage:
    def __init__(self, fid):
        self.fid=fid
        self.all_fidpid=None
        self.all_share_count=None
        self.all_share_label=None
        self.all_created_time=None
        self.all_is_peak_time=None
        self.all_post_type=None
        self.all_has_link=None
    def name(self):
        return query(self.fid,"fanpage").hits[0]['_source']['name']
    def get_all_fidpid(self,refresh=False): # output: list of all fid_pid
        result = []
        if refresh==False and self.all_fidpid!=None:
            result = self.all_fidpid
        else:
            es_post = query(self.fid,'post')
            for post in es_post.hits:
                result.append(post['_source']['id'])
            self.all_fidpid=result
        return result   
    def get_all_share_count(self,refresh=False):
        result = []
        if refresh==False and self.all_share_count!=None:
            result = self.all_share_count
        else:
            all_fidpid = self.get_all_fidpid()
            for fidpid in all_fidpid:
                share_count = get_share_count(fidpid)
                result.append(share_count)
            self.all_share_count = result
        return result
    def get_all_share_label(self,refresh=False):
        result = []
        if refresh==False and self.all_share_label!=None:
            result = self.all_share_label
        else:
            all_share_count = self.get_all_share_count()
            for share_count in all_share_count:
                if share_count>0:
                    result.append(1)
                else:
                    result.append(0)
            self.all_share_label = result
        return result
    def get_all_created_time(self,refresh=False):
        result = []
        if refresh==False and self.all_created_time!=None:
            result = self.all_created_time
        else:
            for fidpid in self.get_all_fidpid():
                created_time = get_created_time(fidpid)
                result.append(created_time) 
            self.all_created_time = result
        return result
    def get_all_is_peak_time(self,refresh=False):
        result = []
        if refresh==False and self.all_is_peak_time!=None:
            result = self.all_is_peak_time
        else:
            for created_time in self.get_all_created_time():
                datetime_obj = datetime.strptime(created_time[11:16], '%H:%M').time()
                if datetime_obj>=time(16,0) or datetime_obj<=time(4,0):
                    result.append(1)
                else:
                    result.append(0)
            self.all_is_peak_time = result
        return result
    def get_all_post_type(self,refresh=False):
        result = []
        if refresh==False and self.all_post_type!=None:
            result = self.all_post_type
        else:
            all_fidpid = self.get_all_fidpid()
            for fidpid in all_fidpid:
                post_type = get_type(fidpid)
                if post_type =='link':
                    result.append(1)
                elif post_type =='photo':
                    result.append(2)
                elif post_type =='video':
                    result.append(3)
                else:
                    result.append(0)
            self.all_post_type = result
        return result
    def get_all_has_link(self,refresh=False):
        result = []
        if refresh==False and self.all_has_link!=None:
            result = self.all_has_link
        else:
            for fidpid in self.get_all_fidpid():
                has_link_status = has_link(fidpid)
                result.append(has_link_status) 
            self.all_has_link = result
        return result

In [11]:
fp_samsung = fanpage('131307770301236')

In [12]:
print fp_samsung.name()

屈臣氏 Watsons Taiwan


In [13]:
fp_samsung.get_all_share_label()

[1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [16]:
fp_samsung.get_all_is_peak_time()

[0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [17]:
fp_samsung.get_all_post_type()

[2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 1,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 3,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 3,
 2,
 3,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 3,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 3,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,


In [18]:
fp_samsung.get_all_has_link()

[1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,


## 將資料分成test data 和train data

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
import numpy as np

In [21]:
s_train_X,s_test_X,s_train_y,s_test_y=train_test_split(fp_samsung.all_fidpid,fp_samsung.all_share_label,train_size=0.5,random_state=123,stratify=fp_samsung.all_share_label)

In [22]:
s_Y = s_train_y+s_test_y
print('all:',np.bincount(s_Y)/float(len(s_Y)))
print('Test:', np.bincount(s_test_y) / float(len(s_test_y)) * 100.0)
print('Train:', np.bincount(s_train_y) / float(len(s_train_y)) * 100.0)

('all:', array([ 0.07388917,  0.92611083]))
('Test:', array([  7.38891663,  92.61108337]))
('Train:', array([  7.38891663,  92.61108337]))


# 選取keywords當作資料特徵
### 把po文整合成一個大字串，分成share, unshare

In [23]:
import jieba
import jieba.analyse

In [24]:
jieba.set_dictionary("/Users/ning_kang/Desktop/school/social_media_analysis/dict.txt.big")
stop_words_path = "/Users/ning_kang/Desktop/school/social_media_analysis/w_stop_words.txt"
jieba.analyse.set_stop_words(stop_words_path)

In [25]:
def get_post_message(fidpid):
    tq = pyes.query.TermQuery(field="id",value=fidpid)
    qsearch = pyes.query.Search(tq)
    result = conn.search(query=qsearch , doc_types="post")
    try:
        message = result.hits[0]["_source"]["message"]
    except:
        message = ''
    return message

In [26]:
# 輸入fid_pid的list, 輸出所有po文彙整
def get_combined_message(fid_pid_list):
    combined_result = ""
    for i in fid_pid_list:
        combined_result += get_post_message(i[0])
    return combined_result

In [27]:
nd_s_train_X = np.array(s_train_X)[:,np.newaxis]
nd_s_train_y = np.array(s_train_y)

In [28]:
s_share_combined_message = get_combined_message(nd_s_train_X[nd_s_train_y==1,:])

In [29]:
s_unshare_combined_message = get_combined_message(nd_s_train_X[nd_s_train_y==0,:])

In [30]:
s_share_keywords = jieba.analyse.extract_tags(s_share_combined_message, 30)
s_unshare_keywords = jieba.analyse.extract_tags(s_unshare_combined_message, 30)

Building prefix dict from /Users/ning_kang/Desktop/school/social_media_analysis/dict.txt.big ...
Loading model from cache /var/folders/q1/sy8zj9z12v792k86g84xby1r0000gn/T/jieba.udb8ee2b6cd8f3b94b12a64cf1481654e.cache
Loading model cost 0.770 seconds.
Prefix dict has been built succesfully.


In [31]:
samsung_keywords=list(set(s_share_keywords+s_unshare_keywords))

In [32]:
print ','.join(samsung_keywords)

保濕,參見,小編,中獎,獨家,公佈,商品,FB,個人資料,優惠,id,幸運,活動,小屈,獎品,粉絲,權利,留言,mail,BANAO,分享,肌膚,美麗,獲得,確認,得獎,名單,寄送,現在,facebook,資料,真實,健康,機會,保養,彩妝,look,獎項,說明,蒐集,名稱,手機,屈臣氏


# 建構feature matrix

In [33]:
import math

In [34]:
import pandas as pd

In [35]:
# 輸入fidpid, 輸出list:[1,0,0,....,2,0]記錄關鍵字出現次數
def count_keywords(fidpid,keywords):
    result = []
    message = get_post_message(fidpid)
    words = [w for w in jieba.cut(message, cut_all=False)]
    for k in keywords:
        count = 0
        for w in words:
            if k==w:
#                 print w
                count += 1
        result.append(count)
    return result

In [36]:
def build_matrix(fp_obj, fidpid_list, keywords):
    matrix=[]
    for i in fidpid_list:
        fidpid_feature=None
        for fidpid,peak_time,post_type,has_link in zip(fp_obj.all_fidpid,
                                                       fp_obj.all_is_peak_time,
                                                       fp_obj.all_post_type,
                                                       fp_obj.all_has_link):
            if i==fidpid:
                keyword_freq_list=count_keywords(fidpid,keywords)
                fidpid_feature=[fidpid,peak_time,post_type,has_link]+keyword_freq_list
                break
        matrix.append(fidpid_feature)
    return matrix

In [37]:
s_train_X_matrix = build_matrix(fp_samsung,s_train_X,samsung_keywords)

In [38]:
s_test_X_matrix = build_matrix(fp_samsung,s_test_X,samsung_keywords)

In [39]:
pd.DataFrame(data=s_train_X_matrix,
             columns=['fid_pid','peak_time','post_type','has_link']+samsung_keywords).iloc[0:10]

,fid_pid,peak_time,post_type,has_link,保濕,參見,小編,中獎,獨家,公佈,...,機會,保養,彩妝,look,獎項,說明,蒐集,名稱,手機,屈臣氏
0,131307770301236_876462012452471,0,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,131307770301236_831856840246322,1,2,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2,131307770301236_506582659440410,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,131307770301236_501152486650094,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,131307770301236_815814225183917,1,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,131307770301236_665908383507836,0,2,1,0,0,2,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6,131307770301236_613152268783448,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,131307770301236_899796813452324,0,2,0,1,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
8,131307770301236_725822460849761,0,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,131307770301236_720667208031953,0,2,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
s_train_X_matrix_cut = [i[1:] for i in s_train_X_matrix]

In [41]:
s_test_X_matrix_cut = [i[1:] for i in s_test_X_matrix]

In [42]:
type(s_test_X_matrix_cut)

list

# Machine Learning Time

In [43]:
from sklearn.model_selection import cross_val_score

In [44]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA 

In [85]:
class ml_dataset:
    def __init__(self, train_X, test_X, train_y, test_y):  # dtype : list
        self.train_X=train_X
        self.test_X=test_X
        self.train_y=train_y
        self.test_y=test_y
        self.lg_accuracy = None
        self.lg_cross_val_score_mean = None
        self.lg_cross_val_score_std = None
        self.nb_accuracy = None
        self.nb_cross_val_score_mean = None
        self.nb_cross_val_score_std = None
        self.hb_accuracy = None
        self.hb_cross_val_score_mean = None
        self.hb_cross_val_score_std = None
        self.pca_accuracy = None
        self.pca_cross_val_score_mean = None
        self.pca_cross_val_score_std = None
    def logistic_regression(self,refresh=False):
        if refresh==False and self.lg_accuracy is not None:
            return self.lg_accuracy,self.lg_cross_val_score_mean,self.lg_cross_val_score_std
        else:
            enc = OneHotEncoder(categorical_features=[0,1,2],handle_unknown='error', n_values='auto', sparse=True)
            enc.fit(self.train_X)
            train_X_onehot = enc.transform(self.train_X).toarray()
            test_X_onehot = enc.transform(self.test_X).toarray()
            lg = LogisticRegression()
            lg.fit(train_X_onehot, self.train_y)
            prediction = lg.predict(test_X_onehot)
            accuracy = np.mean(prediction==self.test_y)
            scores = cross_val_score(lg, train_X_onehot.tolist()+test_X_onehot.tolist(), self.train_y+self.test_y, cv=5)
            cross_val_score_mean = scores.mean()
            cross_val_score_std = scores.std() * 2
            self.lg_accuracy = accuracy
            self.lg_cross_val_score_mean = cross_val_score_mean
            self.lg_cross_val_score_std = cross_val_score_std
            return accuracy, cross_val_score_mean,cross_val_score_std
    def naive_bayes(self,refresh=False):
        if refresh==False and self.nb_accuracy is not None:
            return self.nb_accuracy,self.nb_cross_val_score_mean,self.nb_cross_val_score_std
        else:
            enc = OneHotEncoder(categorical_features=[0,1,2],handle_unknown='error', n_values='auto', sparse=True)
            enc.fit(self.train_X)
            train_X_onehot = enc.transform(self.train_X).toarray()
            test_X_onehot = enc.transform(self.test_X).toarray() 
            gnb = GaussianNB()
            gnb.fit(train_X_onehot,self.train_y)  
            prediction = gnb.predict(test_X_onehot)
            accuracy = np.mean(prediction==self.test_y)
            scores = cross_val_score(gnb, train_X_onehot.tolist()+test_X_onehot.tolist(), self.train_y+self.test_y, cv=5)
            cross_val_score_mean = scores.mean()
            cross_val_score_std = scores.std() * 2
            self.nb_accuracy = accuracy
            self.nb_cross_val_score_mean = cross_val_score_mean
            self.nb_cross_val_score_std = cross_val_score_std
            return accuracy, cross_val_score_mean,cross_val_score_std
    def make_matrix_new(self,X_matrix_category, X_key_pred):
        result = []
        for category_feature,pred_feature in zip(X_matrix_category, X_key_pred):
            result.append(category_feature+pred_feature.tolist())
        return result
    def hybrid(self,refresh=False):
        if refresh==False and self.hb_accuracy is not None:
            return self.hb_accuracy,self.hb_cross_val_score_mean,self.hb_cross_val_score_std
        else:
            train_X_matrix_category = [i[0:3] for i in self.train_X] 
            test_X_matrix_category = [i[0:3] for i in self.test_X]
            train_X_matrix_keyword = [i[3:] for i in self.train_X]
            test_X_matrix_keyword = [i[3:] for i in self.test_X]
            gnb = GaussianNB()
            gnb.fit(train_X_matrix_keyword, self.train_y)
            train_X_key_pred = [gnb.predict([i]) for i in train_X_matrix_keyword]
            test_X_key_pred = [gnb.predict([i]) for i in test_X_matrix_keyword]
            train_X_matrix_new = self.make_matrix_new(train_X_matrix_category,train_X_key_pred)
            test_X_matrix_new = self.make_matrix_new(test_X_matrix_category,test_X_key_pred)
            enc = OneHotEncoder(categorical_features=[0,1,2,3],handle_unknown='error', n_values='auto', sparse=True)
            enc.fit(train_X_matrix_new)
            train_matrix_new_onehot = enc.transform(train_X_matrix_new).toarray()
            test_matrix_new_onehot = enc.transform(test_X_matrix_new).toarray()
            lg = LogisticRegression()
            lg.fit(train_matrix_new_onehot, self.train_y)
            prediction = lg.predict(test_matrix_new_onehot)
            accuracy = np.mean(prediction==self.test_y)
            scores = cross_val_score(lg, train_matrix_new_onehot.tolist()+test_matrix_new_onehot.tolist(), self.train_y+self.test_y, cv=5)
            cross_val_score_mean = scores.mean()
            cross_val_score_std = scores.std() * 2
            self.hb_accuracy = accuracy
            self.hb_cross_val_score_mean = cross_val_score_mean
            self.hb_cross_val_score_std = cross_val_score_std
            return accuracy, cross_val_score_mean,cross_val_score_std
    def PCA_lg(self,refresh=False):
        if refresh==False and self.pca_accuracy is not None:
            return self.pca_accuracy,self.pca_cross_val_score_mean,self.pca_cross_val_score_std
        else:
            pca=PCA(n_components=3)
            pca.fit_transform(s_train_X_matrix_cut)
            train_X_PCA=pca.transform(self.train_X)
            test_X_PCA=pca.transform(self.test_X)
            lg = LogisticRegression()
            lg.fit(train_X_PCA, self.train_y)
            prediction = lg.predict(test_X_PCA)
            accuracy = np.mean(prediction==self.test_y)
            scores = cross_val_score(lg, train_X_PCA.tolist()+test_X_PCA.tolist(), self.train_y+self.test_y, cv=5)
            cross_val_score_mean = scores.mean()
            cross_val_score_std = scores.std() * 2
            self.pca_accuracy = accuracy
            self.pca_cross_val_score_mean = cross_val_score_mean
            self.pca_cross_val_score_std = cross_val_score_std
            return accuracy, cross_val_score_mean,cross_val_score_std

## original_data (imbalanced)

In [86]:
original_data = ml_dataset(s_train_X_matrix_cut,s_test_X_matrix_cut,s_train_y,s_test_y)

In [87]:
print len(original_data.train_y)
print 's_train_y 比例:',np.bincount(original_data.train_y)/float(len(original_data.train_y))*100

2003
s_train_y 比例: [  7.38891663  92.61108337]


### Discriminative Modeling - Logistic Regression

In [88]:
original_data.logistic_regression()

(0.92710933599600598, 0.92661044019165573, 0.0028741075024283137)

In [89]:
print "test_data_accuracy : ", original_data.lg_accuracy
print "cross_validate_accuracy: %0.2f (+/- %0.2f)" % (original_data.lg_cross_val_score_mean, original_data.lg_cross_val_score_std * 2)

test_data_accuracy :  0.927109335996
cross_validate_accuracy: 0.93 (+/- 0.01)


### Generative Modeling - Naive Bayes

In [90]:
original_data.naive_bayes()

(0.8132800798801797, 0.83349678238859792, 0.018888219932201744)

In [91]:
print "test_data_accuracy : ", original_data.nb_accuracy
print "cross_validate_accuracy: %0.2f (+/- %0.2f)" % (original_data.nb_cross_val_score_mean, original_data.nb_cross_val_score_std * 2)

test_data_accuracy :  0.81328007988
cross_validate_accuracy: 0.83 (+/- 0.04)


### hybrid

In [92]:
original_data.hybrid()

(0.92611083374937597, 0.92586137652124378, 0.0025253562132918502)

In [93]:
print "test_data_accuracy : ", original_data.hb_accuracy
print "cross_validate_accuracy: %0.2f (+/- %0.2f)" % (original_data.hb_cross_val_score_mean, original_data.hb_cross_val_score_std * 2)

test_data_accuracy :  0.926110833749
cross_validate_accuracy: 0.93 (+/- 0.01)


### PCA 降維

In [94]:
original_data.PCA_lg()

(0.92611083374937597, 0.92611106441138102, 0.0009240319924284534)

In [95]:
print "test_data_accuracy : ", original_data.pca_accuracy
print "cross_validate_accuracy: %0.2f (+/- %0.2f)" % (original_data.pca_cross_val_score_mean, original_data.pca_cross_val_score_std * 2)

test_data_accuracy :  0.926110833749
cross_validate_accuracy: 0.93 (+/- 0.00)


# Oversampling -- Balanced Training Data

In [105]:
from imblearn.over_sampling import SMOTE

In [106]:
# Apply regular SMOTE
sm = SMOTE(kind='regular')
X_train_resampled, y_train_resampled = sm.fit_sample(s_train_X_matrix_cut, s_train_y)

In [107]:
print len(y_train_resampled)
print 's_train_y 比例:',np.bincount(y_train_resampled)/float(len(y_train_resampled))*100

3710
s_train_y 比例: [ 50.  50.]


In [108]:
balanced_data = ml_dataset(X_train_resampled.tolist(),s_test_X_matrix_cut,y_train_resampled.tolist(),s_test_y)

### Discriminative Modeling - Logistic Regression

In [114]:
balanced_data.logistic_regression()

(0.64503245132301545, 0.73078848944232233, 0.02795834681093692)

In [115]:
print "test_data_accuracy : ", balanced_data.lg_accuracy
print "cross_validate_accuracy: %0.2f (+/- %0.2f)" % (balanced_data.lg_cross_val_score_mean, balanced_data.lg_cross_val_score_std * 2)

test_data_accuracy :  0.645032451323
cross_validate_accuracy: 0.73 (+/- 0.06)


### Generative Modeling - Naive Bayes

In [116]:
balanced_data.naive_bayes()
print "test_data_accuracy : ", balanced_data.nb_accuracy
print "cross_validate_accuracy: %0.2f (+/- %0.2f)" % (balanced_data.nb_cross_val_score_mean, balanced_data.nb_cross_val_score_std * 2)

test_data_accuracy :  0.6445332002
cross_validate_accuracy: 0.69 (+/- 0.08)


### hybrid

In [117]:
balanced_data.hybrid()
print "test_data_accuracy : ", balanced_data.hb_accuracy
print "cross_validate_accuracy: %0.2f (+/- %0.2f)" % (balanced_data.hb_cross_val_score_mean, balanced_data.hb_cross_val_score_std * 2)

test_data_accuracy :  0.631552670994
cross_validate_accuracy: 0.72 (+/- 0.05)


### PCA 降維

In [118]:
balanced_data.PCA_lg()
print "test_data_accuracy : ", balanced_data.pca_accuracy
print "cross_validate_accuracy: %0.2f (+/- %0.2f)" % (balanced_data.pca_cross_val_score_mean, balanced_data.pca_cross_val_score_std * 2)

test_data_accuracy :  0.531203195207
cross_validate_accuracy: 0.68 (+/- 0.03)
